In [ ]:
# 安装依赖
! pip install tokenizers
! pip install sentencepiece
! pip install datasets

# 训练一个自己的tokenizer（走通流程）

## 准备训练tokenizer的数据

In [ ]:
import datasets
# 准备所有的语料
# 应该是按照某些比例做语种、cate的采样，这里为了简单，就是中文、英文各若干个。

sample_size = 10000
TMP_FILE = "tmp_file.txt"

# 英语
dataset_en = datasets.load_dataset("wikitext", "wikitext-103-raw-v1", split="train+test+validation")
# 中文
dataset_cn = datasets.load_dataset("m-a-p/MAP-CC", split='train', streaming=True)

all_text = []
for index, d in enumerate(dataset_en, 1):
    if index > sample_size:
        break
    all_text.append(d["text"])
for index, d in enumerate(dataset_cn, 1):
    if index > sample_size:
        break
    all_text.append(d["text"])

with open(TMP_FILE, "w") as f:
    for text in all_text:
        # 去除首尾的空字符
        print(text.strip(), file=f)


## 用sentencepiece进行tokenizer的训练

In [ ]:
import sentencepiece as spm
import os

spm.SentencePieceTrainer.train(
                                input=TMP_FILE,
                                model_prefix="spm_model",
                                model_type="bpe",
                                vocab_size=10000,
                                self_test_sample_size=0,
                                input_format="text",
                                character_coverage=1.0,
                                num_threads=os.cpu_count(),
                                split_digits=True,
                                allow_whitespace_only_pieces=True,
                                byte_fallback=True,
                                normalization_rule_name="identity"
                            )

## 用tokenizers进行tokenizer的训练

In [ ]:
from tokenizers import Tokenizer, pre_tokenizers, models, trainers
from tokenizers.pre_tokenizers import Digits, Whitespace


tokenizer = Tokenizer(models.BPE())
# 不做normalizion
# tokenizer.normalizer = normalizer
pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Digits(individual_digits=True)])
tokenizer.pre_tokenizer = pre_tokenizer

trainer = trainers.BpeTrainer(
    vocab_size=10000,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
    special_tokens=["<PAD>", "<BOS>", "<EOS>"],
)

tokenizer.train([TMP_FILE], trainer=trainer)
tokenizer.save("tokenizer.json")


# 一些总结

1. 训练一个多语的tokenizer时候，需要把所有语种、每个语种领域（丰富性）都有准备，同时按照占比进行对齐。
2. 通过观察目前开源模型的tokenzier，可以发现：
   - vocab_size一般订一个大概范围（根据已有的实验/其他tokenizer），然后再根据训练时分布式训练的并行考虑，尽量保证。
   - 对于标点符号、HTML中的tag、预保留的token都是通过user_defined_symbols预先定义的（baichuan-inc/Baichuan-7B中前几百个字符可以观察出）。
   - 一般要设置将数字都进行切分，这样对于数学能力有提升。——引申出来，是否可以将latex、markdown的一些符号也进行切分，这样对这些的学习也会更好？
3. 自己训练一个好的tokenizer是比较难的，可以使用开源的+自己领域的进行增量训练。